In [1]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")


In [14]:
# import torch
# from transformers import BitsAndBytesConfig, AutoModel

# bnb_config = BitsAndBytesConfig(
# load_in_4bit=True,
# bnb_4bit_use_double_quant=True,
# bnb_4bit_quant_type="nf4",
# bnb_4bit_compute_dtype=torch.bfloat16
# )

# encoder = AutoModel.from_pretrained(
# 'Salesforce/SFR-Embedding-Mistral',
# trust_remote_code=True,
# device_map='auto',
# torch_dtype=torch.bfloat16,
# quantization_config=bnb_config
# )

In [2]:
# #embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
# #embed_model_id = 'voyage-lite-02-instruct' #not available through HF
# embed_model_id = 'Salesforce/SFR-Embedding-Mistral'
# core_embeddings_model = HuggingFaceEmbeddings(
#     model_name=encoder,
#     model_kwargs={"trust_remote_code":True}
#     # model_name=embed_model_id
# )
# embedder = CacheBackedEmbeddings.from_bytes_store(
#     core_embeddings_model, store, namespace=embed_model_id

# )

In [15]:
#embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
#embed_model_id = 'voyage-lite-02-instruct' #not available through HF
#embed_model_id = 'WhereIsAI/UAE-Large-V1'
embed_model_id = 'mixedbread-ai/mxbai-embed-large-v1'
core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={"trust_remote_code":True}
    # model_name=embed_model_id
)
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id

)

In [4]:
import os
existing_files = set(os.listdir('../abstracts'))

In [29]:
#existing_files = list(existing_files)

In [5]:
def get_title_abstract_filename(directory = '../abstracts/',filename=None):
    path = os.path.join(directory, filename)
    with open(path, 'r') as file:
        lines = file.readlines()
        title = lines[0].strip()
        abstract = "".join(lines[1:]).replace('\n', ' ')
    return title, abstract, filename


In [6]:
get_title_abstract_filename(filename=next(iter(existing_files)))

('In-Context Retrieval-Augmented Language Models',
 'Retrieval-Augmented Language Modeling (RALM) methods, which condition a language model (LM) on relevant documents from a grounding corpus during generation, were shown to significantly improve language modeling performance. In addition, they can mitigate the problem of factually inaccurate text generation and provide natural source attribution mechanism. Existing RALM approaches focus on modifying the LM architecture in order to facilitate the incorporation of external information, significantly complicating deployment. This paper considers a simple alternative, which we dub In-Context RALM: leaving the LM architecture unchanged and prepending grounding documents to the input, without any further training of the LM. We show that In-Context RALM that builds on off-the-shelf general purpose retrievers provides surprisingly large LM gains across model sizes and diverse corpora. We also demonstrate that the document retrieval and ranking

In [7]:
for file in existing_files:
    title, abstract,_ = get_title_abstract_filename(filename=file)
    print(title)
    print(abstract)
    print()

In-Context Retrieval-Augmented Language Models
Retrieval-Augmented Language Modeling (RALM) methods, which condition a language model (LM) on relevant documents from a grounding corpus during generation, were shown to significantly improve language modeling performance. In addition, they can mitigate the problem of factually inaccurate text generation and provide natural source attribution mechanism. Existing RALM approaches focus on modifying the LM architecture in order to facilitate the incorporation of external information, significantly complicating deployment. This paper considers a simple alternative, which we dub In-Context RALM: leaving the LM architecture unchanged and prepending grounding documents to the input, without any further training of the LM. We show that In-Context RALM that builds on off-the-shelf general purpose retrievers provides surprisingly large LM gains across model sizes and diverse corpora. We also demonstrate that the document retrieval and ranking mecha

In [8]:
import pandas as pd
new_articles=pd.read_csv('/home/mainuser/Desktop/LLMs/RagOverArXiv/scrape/data/articles_up_to_2024-04-16.csv')

In [9]:
new_articles.head()

,title,authors,abstract,arxiv_abbrev
0,Ferret-UI: Grounded Mobile UI Understanding wi...,"Authors:Keen You,Haotian Zhang,Eldon Schoop,Fl...",Recent advancements in multimodal large langua...,2404.05719
1,"ByteEdit: Boost, Comply and Accelerate Generat...","Authors:Yuxi Ren,Jie Wu,Yanzuo Lu,Huafeng Kuan...",Recent advancements in diffusion-based generat...,2404.04860
2,SpatialTracker: Tracking Any 2D Pixels in 3D S...,"Authors:Yuxi Xiao,Qianqian Wang,Shangzhan Zhan...",Recovering dense and long-range pixel motion i...,2404.04319
3,SwapAnything: Enabling Arbitrary Object Swappi...,"Authors:Jing Gu,Yilin Wang,Nanxuan Zhao,Wei Xi...",Effective editing of personal content holds a ...,2404.05717
4,BeyondScene: Higher-Resolution Human-Centric S...,"Authors:Gwanghyun Kim,Hayeon Kim,Hoigi Seo,Don...",Generating higher-resolution human-centric sce...,2404.04544


In [10]:
from langchain.schema.document import Document

In [11]:
docs = [get_title_abstract_filename(filename=file) for file in existing_files]
docs = [Document(page_content=doc[1],metadata={'source':doc[0], 'filename':doc[2]}) for doc in docs]


In [12]:
len(docs)

37

In [15]:
dir(docs[0])

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose_class',
 '_enfo

In [16]:
docs[0].__getattribute__('page_content')

"Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches. Specifically, we train GPT-3, an autoregressive language model with 175 billion parameters, 10x more than any previous non-sparse language model, and test its performance in the few-shot setting. For all tasks, GPT-3 is applied without any gradient updates or fine-tuning, with tasks and few-shot demons

In [50]:
len(docs)

37

In [16]:
for i, doc in enumerate(docs):
   #doc = doc[1]
   #content = docs[index]
   if i == 0:
       vector_store = FAISS.from_documents([doc], embedder)
   else:
      vector_store_i = FAISS.from_documents([doc], embedder)
      vector_store.merge_from(vector_store_i)

vector_store

In [60]:
#dir(vector_store)

In [17]:
len(vector_store.index_to_docstore_id)

37

In [53]:
#vector_store.asimilaritysearch('What is PPO?')
vector_store.asimilarity_search('What is PPO?')

<coroutine object FAISS.asimilarity_search at 0x7809be41d480>

In [18]:
query = "The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straightforward update, we demonstrate the effectiveness of TR-DPO against DPO on the Anthropic HH and TLDR datasets. We show that TR-DPO outperforms DPO by up to 19%, measured by automatic evaluation with GPT-4. The new alignment approach that we propose allows us to improve the quality of models across several parameters at once, such as coherence, correctness, level of detail, helpfulness, and harmlessness."
#query = 'The quadratic complexity and weak length extrapolation of Transformers limits their ability to scale to long sequences, and while sub-quadratic solutions like linear attention and state space models exist, they empirically underperform Transformers in pretraining efficiency and downstream task accuracy. We introduce Megalodon, a neural architecture for efficient sequence modeling with unlimited context length. Megalodon inherits the architecture of Mega (exponential moving average with gated attention), and further introduces multiple technical components to improve its capability and stability, including complex exponential moving average (CEMA), timestep normalization layer, normalized attention mechanism and pre-norm with two-hop residual configuration. In a controlled head-to-head comparison with Llama2, Megalodon achieves better efficiency than Transformer in the scale of 7 billion parameters and 2 trillion training tokens. Megalodon reaches a training loss of 1.70, landing mid-way between Llama2-7B (1.75) and 13B (1.67). Code: https://github.com/XuezheMax/megalodon'
#query = ''
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)
  print(page.metadata)

While large-scale unsupervised language models (LMs) learn broad world knowledge and some reasoning skills, achieving precise control of their behavior is difficult due to the completely unsupervised nature of their training. Existing methods for gaining such steerability collect human labels of the relative quality of model generations and fine-tune the unsupervised LM to align with these preferences, often with reinforcement learning from human feedback (RLHF). However, RLHF is a complex and often unstable procedure, first fitting a reward model that reflects the human preferences, and then fine-tuning the large unsupervised LM using reinforcement learning to maximize this estimated reward without drifting too far from the original model. In this paper we introduce a new parameterization of the reward model in RLHF that enables extraction of the corresponding optimal policy in closed form, allowing us to solve the standard RLHF problem with only a simple classification loss. The resu

In [62]:
query = "The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straightforward update, we demonstrate the effectiveness of TR-DPO against DPO on the Anthropic HH and TLDR datasets. We show that TR-DPO outperforms DPO by up to 19%, measured by automatic evaluation with GPT-4. The new alignment approach that we propose allows us to improve the quality of models across several parameters at once, such as coherence, correctness, level of detail, helpfulness, and harmlessness."
source = 'Learn Your Reference Model for Real Good Alignment'
filename = '2404.09656.txt'
new_doc = Document(page_content=query,metadata={'source':source, 'filename':filename})
vector_store_i = FAISS.from_documents([new_doc], embedder)
vector_store.merge_from(vector_store_i)

In [63]:
len(vector_store.index_to_docstore_id)

38

In [64]:
query = "The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straightforward update, we demonstrate the effectiveness of TR-DPO against DPO on the Anthropic HH and TLDR datasets. We show that TR-DPO outperforms DPO by up to 19%, measured by automatic evaluation with GPT-4. The new alignment approach that we propose allows us to improve the quality of models across several parameters at once, such as coherence, correctness, level of detail, helpfulness, and harmlessness."
#query = 'The quadratic complexity and weak length extrapolation of Transformers limits their ability to scale to long sequences, and while sub-quadratic solutions like linear attention and state space models exist, they empirically underperform Transformers in pretraining efficiency and downstream task accuracy. We introduce Megalodon, a neural architecture for efficient sequence modeling with unlimited context length. Megalodon inherits the architecture of Mega (exponential moving average with gated attention), and further introduces multiple technical components to improve its capability and stability, including complex exponential moving average (CEMA), timestep normalization layer, normalized attention mechanism and pre-norm with two-hop residual configuration. In a controlled head-to-head comparison with Llama2, Megalodon achieves better efficiency than Transformer in the scale of 7 billion parameters and 2 trillion training tokens. Megalodon reaches a training loss of 1.70, landing mid-way between Llama2-7B (1.75) and 13B (1.67). Code: https://github.com/XuezheMax/megalodon'
#query = ''
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)
  print(page.metadata)

The complexity of the alignment problem stems from the fact that existing methods are unstable. Researchers continuously invent various tricks to address this shortcoming. For instance, in the fundamental Reinforcement Learning From Human Feedback (RLHF) technique of Language Model alignment, in addition to reward maximization, the Kullback-Leibler divergence between the trainable policy and the SFT policy is minimized. This addition prevents the model from being overfitted to the Reward Model (RM) and generating texts that are out-of-domain for the RM. The Direct Preference Optimization (DPO) method reformulates the optimization task of RLHF and eliminates the Reward Model while tacitly maintaining the requirement for the policy to be close to the SFT policy. In our paper, we argue that this implicit limitation in the DPO method leads to sub-optimal results. We propose a new method called Trust Region DPO (TR-DPO), which updates the reference policy during training. With such a straig

In [22]:
new_articles.loc[0,'abstract']

'Recent advancements in multimodal large language models (MLLMs) have been noteworthy, yet, these general-domain MLLMs often fall short in their ability to comprehend and interact effectively with user interface (UI) screens. In this paper, we present Ferret-UI, a new MLLM tailored for enhanced understanding of mobile UI screens, equipped with referring, grounding, and reasoning capabilities. Given that UI screens typically exhibit a more elongated aspect ratio and contain smaller objects of interest (e.g., icons, texts) than natural images, we incorporate "any resolution" on top of Ferret to magnify details and leverage enhanced visual features. Specifically, each screen is divided into 2 sub-images based on the original aspect ratio (i.e., horizontal division for portrait screens and vertical division for landscape screens). Both sub-images are encoded separately before being sent to LLMs. We meticulously gather training samples from an extensive range of elementary UI tasks, such as

In [21]:
query=new_articles.loc[0,'abstract']
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)
  print(page.metadata)

We continue the investigation into the power of smaller Transformer-based language models as initiated by \textbf{TinyStories} -- a 10 million parameter model that can produce coherent English -- and the follow-up work on \textbf{phi-1}, a 1.3 billion parameter model with Python coding performance close to the state-of-the-art. The latter work proposed to use existing Large Language Models (LLMs) to generate ``textbook quality" data as a way to enhance the learning process compared to traditional web data. We follow the ``Textbooks Are All You Need" approach, focusing this time on common sense reasoning in natural language, and create a new 1.3 billion parameter model named \textbf{phi-1.5}, with performance on natural language tasks comparable to models 5x larger, and surpassing most non-frontier LLMs on more complex reasoning tasks such as grade-school mathematics and basic coding. More generally, \textbf{phi-1.5} exhibits many of the traits of much larger LLMs, both good -- such as 

In [35]:
# for i, doc in enumerate(docs):
#    #content = docs[index]
#    if i == 0:
#        vector_store = FAISS.from_documents(doc.__getattribute__('page_content'), embedder)
#    else:
#       vector_store_i = FAISS.from_documents(doc.__getattribute__('page_content'), embedder)
#       vector_store.merge_from(vector_store_i)

# vector_store

In [36]:
# for index, doc in enumerate(docs):
#    #content = docs[index]
#    if index == 0:
#        vector_store = FAISS.from_texts(doc.page_content, embedder)
#    else:
#       vector_store_i = FAISS.from_texts(doc.page_content, embedder)
#       vector_store.merge_from(vector_store_i)

# vector_store